A Random Forest regression model will be trained to predict epigenetic age using DNA methylation beta values. 

In [ ]:
#Load require packages
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
# Load data set
GSE55763_beta = pd.read_csv("subset_500_Beta.csv")

In [ ]:
#Display the first rows of the data set
GSE55763_beta.head(3)

In [ ]:
print(GSE55763_beta.shape) #dimensions of the data set

The data set contains 2664 rows and 503 columns 

In [ ]:
# Select CpG columns
cpg_cols = [col for col in GSE55763_beta.columns if col.startswith("cg")]

# Define predictors and outcome
X = GSE55763_beta[cpg_cols + ["gender"]].copy()
y = GSE55763_beta["age"]

# Encode gender numerically
X["gender"] = X["gender"].map({"M": 1, "F": 0})

In [ ]:
# Split into training and testing sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=500
)

# Scale features
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

Train Random Forest model

In [ ]:
rf = RandomForestRegressor(n_estimators=300, random_state=500)
rf.fit(X_train_scaled, y_train)

Metrics (R2, MAE, RMSE)

In [ ]:
# Predict on test set
y_pred = rf.predict(X_test_scaled)

# Metrics
r2 = rf.score(X_test_scaled, y_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R²: {r2:.2f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

R² of 0.59 indicates that approximately 59% of the variance in chronological age is explained by the model. MAE of 5.07 years shows that, on average, the predicted age deviates from the actual age by about 5 years. RMSE of 6.25 years reflects the overall magnitude of prediction errors, giving slightly more weight to larger deviations.

In this part, SHAP values are computed to interpret the Random Forest model, showing how much each feature (CpG or covariate) contributes to increasing or decreasing the predicted age for each sample.

Summary plot for the test set: tvisualizes the impact of each CpG on the model’s predictions

In [ ]:
# Compute SHAP values on a subset of the test set
explainer = shap.TreeExplainer(rf)
X_shap = X_test_scaled.sample(100, random_state=500)
shap_values = explainer.shap_values(X_shap)

# SHAP summary bar plot
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_shap, plot_type="bar", color="coral", show=False)
plt.title("SHAP Summary Plot (Subset of Test Set)")
plt.tight_layout()
plt.savefig("shap_summary_test_subset.png", dpi=300)
plt.show()

Summary plot (bar plot): shows the average contribution of each CpG to the Random Forest predictions, ranking them by importance using SHAP values.

In [ ]:
# Compute SHAP values on the full test set
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test_scaled)

# SHAP beeswarm summary plot (default style)
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_test_scaled, show=False)
plt.title("SHAP Summary Plot (Test Set)")
plt.tight_layout()
plt.savefig("shap_summary_test.png", dpi=300)
plt.show()
